**步驟 1: 定義數據集**
使用之前提供的超市交易資料集：

In [2]:
from itertools import combinations

transactions = [
    {"牛奶", "麵包", "奶酪"},  # 交易1
    {"牛奶", "麵包"},          # 交易2
    {"牛奶", "咖啡"},          # 交易3
    {"麵包", "奶酪"},          # 交易4
    {"牛奶", "麵包", "咖啡"},  # 交易5
]

**步驟 2: 生成所有的候選項集(C1)**
我們需要先生成可能的候選項集，首先從大小為 1 的單個項目開始。這裡可以從每個交易中提取出唯一的商品。

In [3]:
# Step 2: 提取單項集
def get_single_itemsets(transactions):
    itemsets = set()
    for transaction in transactions:
        itemsets.update(transaction)
    return [{item} for item in itemsets]

single_itemsets = get_single_itemsets(transactions)
print(single_itemsets)

[{'牛奶'}, {'麵包'}, {'咖啡'}, {'奶酪'}]


步驟 3: 計算每個項集的支持度
接著，我們需要計算每個項集的支持度。支持度代表項集在交易中出現的頻率。

In [4]:
# Step 3: 計算支持度
def calculate_support(itemset, transactions):
    count = sum([1 for transaction in transactions if itemset.issubset(transaction)])
    return count / len(transactions)

# 計算單項集的支持度
min_support = 0.6  # 設定最低支持度閾值
frequent_itemsets = []

for itemset in single_itemsets:
    support = calculate_support(itemset, transactions)
    if support >= min_support:
        frequent_itemsets.append((itemset, support))

print(frequent_itemsets)

[({'牛奶'}, 0.8), ({'麵包'}, 0.8)]


步驟 4:生成更大的候選項集(C2, C3, ...)
接著要生成更大的項集(例如, 2個、3個商品的組合)。這可以通過頻繁的項集進行自連接生成(self-join), 並篩選出那些支持度大於等於設定值的組合。

In [5]:
# Step 4: 合併生成更大的項集
from itertools import combinations

def generate_candidate_itemsets(frequent_itemsets, k):
    # 合併項集，生成k項集
    items = set()
    for itemset, _ in frequent_itemsets:
        items.update(itemset)
    return [set(combo) for combo in combinations(items, k)]

# 生成二項集
candidate_2_itemsets = generate_candidate_itemsets(frequent_itemsets, 2)
print(candidate_2_itemsets)

[{'牛奶', '麵包'}]


步驟 5: 計算更大項集的支持度並篩選
現在，對於每一個候選的二項集進行支持度計算，並保留那些滿足最小支持度要求的項集。

In [6]:
# Step 5: 計算二項集的支持度
frequent_2_itemsets = []

for itemset in candidate_2_itemsets:
    support = calculate_support(itemset, transactions)
    if support >= min_support:
        frequent_2_itemsets.append((itemset, support))

print(frequent_2_itemsets)

[({'牛奶', '麵包'}, 0.6)]


步驟 6:生成關聯規則並計算信心度、提升度
在生成頻繁項集之後，我們可以根據這些項集來生成關聯規則。每個規則形式為 A -> B, 表示如果購買了 A, 則可能會購買 B。對每個規則, 我們要計算信心度和提升度。

In [7]:
# Step 6: 計算信心度和提升度
def calculate_confidence(itemset_A, itemset_B, transactions):
    support_A = calculate_support(itemset_A, transactions)
    support_AB = calculate_support(itemset_A.union(itemset_B), transactions)
    return support_AB / support_A

def calculate_lift(itemset_A, itemset_B, transactions):
    support_A = calculate_support(itemset_A, transactions)
    support_B = calculate_support(itemset_B, transactions)
    support_AB = calculate_support(itemset_A.union(itemset_B), transactions)
    return support_AB / (support_A * support_B)

# 對於每個頻繁的二項集，生成規則並計算指標
for itemset, support in frequent_2_itemsets:
    itemset_list = list(itemset)
    for i in range(len(itemset_list)):
        itemset_A = {itemset_list[i]}
        itemset_B = itemset.difference(itemset_A)
        confidence = calculate_confidence(itemset_A, itemset_B, transactions)
        lift = calculate_lift(itemset_A, itemset_B, transactions)
        print(f"規則: {itemset_A} -> {itemset_B}, 信心度: {confidence:.2f}, 提升度: {lift:.2f}")

規則: {'牛奶'} -> {'麵包'}, 信心度: 0.75, 提升度: 0.94
規則: {'麵包'} -> {'牛奶'}, 信心度: 0.75, 提升度: 0.94
